In [31]:
import pandas as pd
import numpy as np

In [32]:
#input ([list of string file names])
def read_csv_files(files):
    #todo:
    """
        1. error handling for incorrect file names
    """
    dfs = []
    #stitch multiple files together (assuming same format)
    for file in files:
        dfs.append(pd.read_csv(file))
    data = pd.concat(dfs, axis=0, ignore_index=True)
    return data.sort_values(by=["date","lkid","analyst","sector"])

In [33]:
#input (dataframe of practical.csv format, flag to return columns with errors)
def validate(data, return_errors=True):
    
    #todo:
    """
        1.confirm requirements for validations (can values be empty? can there be extra columns? duplicate rows?)
        2.add additional validations for scenario as needed
        3.additional date validation (validate actual dates not just date format)
    """
    
    #add columns to flag error rows and provide reason
    data["error"] = False
    data["reasons"] = ""
    
    #all error text edited here instead of in code logic
    reasons = {
        "date": "incorrect date",
        "lkid_ticker": "lkid ticker mismatch",
    }
    
    #date check
    data["date"]= data["date"].apply(str)
    data["reasons"]= np.where(data["date"].str.len() != 8, data["reasons"]+ " "+reasons["date"]+ " |", data["reasons"])
    
    #lkid vs ticker check (example validation, doesn't exist in this data set)
    data["reasons"]= np.where(data["lkid"].str.split("_").str[0] != data["ticker"], data["reasons"]+ " "+reasons["lkid_ticker"]+ 
                              " |", data["reasons"])
    
  
    #flag errors
    data["error"] = np.where(data["reasons"].str.len() !=0 , True, False)
    
    #optionally return data with or without error rows
    #all error rows have reasons attached if process would include manual fix
    if return_errors:
        return data
    
    #for this demo error rows are skipped in the full processing method
    else:
        return data[data["error"]==False]


In [34]:
#input (dataframe of practical.csv format post validation)
def aggregate(data):
    #todo:
    """
        1. check for non empty table after errors rows filtered out
    """
    data = pd.pivot_table(data,
                          index=["date","lkid"],
                          aggfunc={"ticker":"first","analyst":"first","sector":"first","pal":"sum","exposure":"sum"}
                         ).reset_index()
    return data

In [35]:
#input (dataframe of practical.csv format post aggregate, dictionary renames)
def patch_sector_tag(data,renames):
    for rename in renames:
        data["sector"] = np.where(data["sector"] == rename,  renames[rename],  data["sector"])
    return data

In [36]:
#input (dataframe of practical.csv format post aggregate)
def calculate_daily_returns(data):
    #get begin day capital
    capital = pd.pivot_table(data,
                      index=["date"],
                      aggfunc={"exposure":"sum"},
                     )
    capital["begin day capital"] = capital["exposure"].shift(1)
    del capital["exposure"]
    data = pd.merge(left=data,right=capital,on="date",how="left")
    #get begin day capital of first day
    total = data.loc[data["date"] == data.iloc[0]["date"], "exposure"].sum() - data.loc[data["date"] == data.iloc[0]["date"], "pal"].sum()
    data["begin day capital"] = np.where(data["date"] == data.iloc[0]["date"], total , data["begin day capital"])
    data["daily return"] = data["pal"]/data["begin day capital"]
    del data["begin day capital"]
    
    return data

In [47]:
#input ([list of file names], {dictionary of sector renames}, result file name, use error rows flag)
def process(files,renames={},save_name="blotter_new.csv",return_errors=False,show_errors=False):
    file_to_table = read_csv_files(files)
    if show_errors:
        validate(file_to_table, return_errors=True).to_csv(save_name,index=False)
        return 
    table_validated = validate(file_to_table, return_errors)
    table_agg = aggregate(table_validated)
    patched_names = patch_sector_tag(table_agg,renames)
    daily_returns = calculate_daily_returns(patched_names)
    daily_returns.to_csv(save_name,index=False)
    return

In [48]:
#process file as per requirements
process(["practical.csv"],{"Technology":"Information Technology"})

In [51]:
#demo of row validation results
process(["practical.csv"],save_name="errors.csv",show_errors=True)